In [ ]:
HOME_FOLDER = '/kaggle/working'
TEMP_FOLDER = '/kaggle/temp'
update = False

CKPT_FOLDER = f'{HOME_FOLDER}/Fooocus/models/checkpoints'

!mkdir $TEMP_FOLDER

import os
from os import path

%cd $HOME_FOLDER

if not path.exists('Fooocus'):
    get_ipython().system('git clone https://github.com/mashb1t/Fooocus.git')
else:
    get_ipython().system(f'find {CKPT_FOLDER} -maxdepth 1 -type l -delete') # delete any symlinks to temp models from a previous run
    
%cd Fooocus

if update:
    get_ipython().system('git pull')
    
!pip install -q -r requirements_versions.txt 
!pip install -q torch torchvision --force-reinstall --index-url https://download.pytorch.org/whl/cu117
!rm -rf /opt/conda/lib/python3.10/site-packages/pytz-2023.3.dist-info
!rm -rf /opt/conda/lib/python3.10/site-packages/PyYAML-6.0.1.dist-info
!pip install -q pyyaml pytz 
!mamba install openssh -y

In [ ]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = "python $HOME_FOLDER/Fooocus/entry_with_update.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:7865 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            #print(line)
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

In [ ]:
# Step 1: Manually remove the corrupted NumPy files
!rm -rf /opt/conda/lib/python3.10/site-packages/numpy*
!rm -rf /opt/conda/lib/python3.10/site-packages/numpy-1.26.4.dist-info*

# Step 2: Reinstall numpy
!pip install numpy

# Step 3: Verify the installation
!pip show numpy

# Step 4: Upgrade transformers library
!pip install --upgrade transformers

# Step 5: Verify the transformers library installation
import numpy as np
import transformers

print("NumPy version:", np.__version__)
print("Transformers version:", transformers.__version__)

# Step 6: Restart the kernel (do this manually via the Kaggle notebook interface)



In [ ]:
# Step 1: Uninstall the current tokenizers and transformers packages
!pip uninstall -y tokenizers transformers

# Step 2: Install a compatible version of tokenizers
!pip install tokenizers==0.13.3

# Step 3: Install the latest version of transformers compatible with the installed tokenizers
!pip install transformers

# Step 4: Verify the installation of both libraries
import numpy as np
import transformers

print("NumPy version:", np.__version__)
print("Transformers version:", transformers.__version__)

# Step 5: Restart the kernel (do this manually via the Kaggle notebook interface)



# Install Models

In [ ]:
# Install a model to permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"
model_url = 'https://civitai.com/api/download/models/169740?type=Model&format=SafeTensor&size=full&fp=fp16'
model_name = 'ZavyChroma.safetensors'

%cd $CKPT_FOLDER
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

In [ ]:
# Install a LoRA to permanent storage
model_url = 'https://civitai.com/api/download/models/505826'
model_name = 'PaperBook.safetensors'

LORA_FOLDER = HOME_FOLDER + '/Fooocus/models/loras'
%cd $LORA_FOLDER
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

In [ ]:
# Install a model to temporary storage
model_url = 'https://civitai.com/api/download/models/395107?type=Model&format=SafeTensor&size=pruned&fp=fp16'
model_name = 'Unstable.safetensors'

%cd $TEMP_FOLDER
if not path.exists(f'{CKPT_FOLDER}/{model_name}'):
    get_ipython().system(f'wget -O "{model_name}" "{model_url}"')
    get_ipython().system(f'ln -s {TEMP_FOLDER}/{model_name} {CKPT_FOLDER}/')
else:
    print(f'The file {CKPT_FOLDER}/{model_name} already exists')